In [ ]:
# default_exp im2im_annotator

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *

In [ ]:
#export

import json
import textwrap
import uuid
import os
import random
import re

from functools import partial
from math import ceil
from pathlib import Path

from ipycanvas import Canvas, hold_canvas
from ipyevents import Event
from ipywidgets import (AppLayout, VBox, HBox, Button, GridBox, Layout, Checkbox, HTML, IntText, Valid, Output, Image)
from traitlets import Dict, Int, Float, HasTraits, observe, dlink, link, List, Unicode

from ipyannotator.navi_widget import Navi
from ipyannotator.storage import setup_project_paths, get_image_list_from_folder, AnnotationStorage
from ipyannotator.capture_annotator import CaptureGrid
from ipyannotator.image_button import ImageButton
from ipyannotator.bbox_canvas import draw_img


In [ ]:
# export

class ImCanvas(HBox, HasTraits):
    image_path = Unicode()
    _image_scale = Float()
    
    def __init__(self, width=150, height=150):
        
        self._canvas = Canvas(width=width, height=height)

        super().__init__([self._canvas])  

    @observe('image_path')
    def _draw_image(self, change):
        self._image_scale = draw_img(self._canvas, self.image_path, clear=True)

    # Add value as a read-only property
    @property
    def image_scale(self):
        return self._image_scale
    
    def _clear_image(self):
        self._canvas.clear()
        
    # needed to support voila
    # https://ipycanvas.readthedocs.io/en/latest/advanced.html#ipycanvas-in-voila
    def observe_client_ready(self, cb=None):
        self._canvas.on_client_ready(cb)

In [ ]:
im = ImCanvas(35, 35)
im.image_path = '../data/projects/im2im1/class_images/blocks_1.png'
display(im)
im.image_scale

In [ ]:
im._clear_image()
im.image_scale

In [ ]:
#exporti

class Im2ImAnnotatorGUI(AppLayout):
    def __init__(self, im_width=300, im_height=300, 
                       label_width=150, label_height=150, 
                       n_rows=None, n_cols=None, label_autosize=False):
            
        if label_autosize:
            if im_width <100 or im_height < 100:
                label_width = 10
                label_height = 10
            elif im_width >1000 or im_height > 1000:
                label_width = 50
                label_height = 10
            else:
                label_width = min(im_width, im_height)/10
                label_height = label_width
                
        self.label_width = label_width
        self.label_height = label_height
        self.n_rows = n_rows
        self.n_cols = n_cols
        
        self._image = ImCanvas(width=im_width, height=im_height)
            
        self._navi = Navi()
        
        self._save_btn = Button(description="Save",
                                layout=Layout(width='auto'))
        
        
        self._controls_box = HBox([self._navi, self._save_btn],
                                 layout=Layout(display='flex', justify_content='center', flex_flow='wrap', align_items='center'))
        
         
        self._grid_box = CaptureGrid(grid_item=ImageButton, image_width=label_width, image_height=label_height,  n_rows=n_rows, n_cols=n_cols)


        self._grid_label = HTML(value="<b>LABEL</b>",)
        self._labels_box = VBox(children = [self._grid_label, self._grid_box],
                                layout=Layout(display='flex', justify_content='center', flex_wrap='wrap', align_items='center'))

        
        super().__init__(header=None,
                 left_sidebar=VBox([self._image, self._controls_box], layout=Layout(display='flex', justify_content='center', flex_wrap='wrap', align_items='center')),
                 center=self._labels_box,
                 right_sidebar=None,
                 footer=None,
                 pane_widths=(6, 4, 0),
                 pane_heights=(1, 1, 1))
        
    def on_client_ready(self, callback):
        self._image.observe_client_ready(callback)
        

In [ ]:
label_state = {
    '../data/projects/im2im1/class_images/blocks_1.png': {'answer': False}, 
    '../data/projects/im2im1/class_images/blocks_9.png': {'answer': False},
    '../data/projects/im2im1/class_images/blocks_12.png': {'answer': True},
    '../data/projects/im2im1/class_images/blocks_32.png': {'answer': False},
    '../data/projects/im2im1/class_images/blocks_37.png': {'answer': False},
    '../data/projects/im2im1/class_images/blocks_69.png': {'answer': True}
}

In [ ]:
im2im_ = Im2ImAnnotatorGUI(im_height = 500, im_width = 500, label_width=50, label_height=50, n_rows=2, n_cols=3)
im2im_._image.image_path='../data/projects/im2im1/pics/Grass1.png'
im2im_

In [ ]:
im2im_._grid_box.current_state = label_state

In [ ]:
#exporti
def _label_state_to_storage_format(label_state):
    return [Path(k).name for k, v in label_state.items() if v['answer']]

In [ ]:
label_state_storage = _label_state_to_storage_format(label_state); label_state_storage

In [ ]:
#exporti
def _storage_format_to_label_state(storage_format, label_names, label_dir):
    return {str(Path(label_dir)/label): {'answer': label in storage_format} for label in label_names}

In [ ]:
label_names = ['blocks_1.png', 'blocks_9.png', 'blocks_12.png', 'blocks_32.png', 'blocks_37.png', 'blocks_69.png']
restored_label_state = _storage_format_to_label_state(label_state_storage, label_names, '../data/projects/im2im1/class_images/')
test_eq(label_state, restored_label_state)

In [ ]:
import tempfile
tmp_dir = tempfile.TemporaryDirectory(); 

print(tmp_dir.name)

In [ ]:
# dataset generator annotation format

# annotations = {
#     'img_0.jpg': {'labels': [('red', 'rectangle'), ('red', 'rectangle')], 
#                   'bboxs': [(3, 21, 82, 82), (19, 98, 82, 145)]}, 
#     'img_1.jpg': {'labels': [('blue', 'ellipse')], 
#                   'bboxs': [(22, 51, 67, 84)]}, 
#     'img_2.jpg': {'labels': [('yellow', 'ellipse'), ('yellow', 'ellipse'), ('blue', 'rectangle')], 
#                   'bboxs': [(75, 33, 128, 120), (4, 66, 59, 95), (30, 35, 75, 62)]}, 
#     'img_3.jpg': {'labels': [('blue', 'ellipse'), ('red', 'ellipse'), ('yellow', 'ellipse')], 
#                   'bboxs': [(47, 55, 116, 96), (99, 27, 138, 50), (0, 3, 47, 56)]}
# }

In [ ]:
#old ipyannotator annotation format

annotations = {
    str(Path(tmp_dir.name) / 'img_0.jpg'): ['yellow.jpg'],
    str(Path(tmp_dir.name) / 'img_1.jpg'): ['red'],
    str(Path(tmp_dir.name) / 'img_2.jpg'): ['red'],
    str(Path(tmp_dir.name) / 'img_3.jpg'): ['red'],
    str(Path(tmp_dir.name) / 'img_4.jpg'): ['yellow'],
    str(Path(tmp_dir.name) / 'img_5.jpg'): ['yellow'],
    str(Path(tmp_dir.name) / 'img_6.jpg'): ['yellow'],
    str(Path(tmp_dir.name) / 'img_7.jpg'): ['blue'],
    str(Path(tmp_dir.name) / 'img_8.jpg'): ['blue'],
    str(Path(tmp_dir.name) / 'img_9.jpg'): ['yellow']
}

In [ ]:
annot_file = Path(tmp_dir.name)/'annotations.json'
with open(annot_file, 'w') as f:
        json.dump(annotations, f, indent=2)

In [ ]:
#     with open(Path(tmp_dir.name) / 'annotations.json') as json_file:
#         data = json.load(json_file)
#         unique_classes = set(flatten([a['labels'] for a in data.values()]))
#         print(unique_classes)

In [ ]:
# !cat {annot_file}

In [ ]:
#exporti

from PIL import Image, ImageDraw, ImageFont


def text_on_img(text="Hello", lbl_w=None, lbl_h=None, font_size=14, filepath=None):
    font = ImageFont.truetype("lte50712.ttf", font_size)
    
    if lbl_w is None:
        lbl_w = 150
    if lbl_h is None:
        lbl_h = 150
        
    assert(text)
    
    text = text.upper()
    
    ascent, descent = font.getmetrics()

    text_width = font.getmask(text).getbbox()[2]
    text_height = font.getmask(text).getbbox()[3] + descent

    m_width, m_heigth = font.getsize("M")
    char_num_per_line = lbl_w // m_width
    
    image = Image.new(mode = "RGB", size = (lbl_w, lbl_h), color = "white")
    draw = ImageDraw.Draw(image)
     
    words = text.split()
    if len(words) <= 2 and all(font.getsize(w)[0] < lbl_w for w in words):
        t_wrapper = words
    else:
        t_wrapper = textwrap.wrap(text, char_num_per_line)
    
    
    offset = (lbl_h - (m_heigth * len(t_wrapper))) // 2

    for line in t_wrapper:
        line_w, line_h = font.getsize(line)
        draw.text(((lbl_w - line_w) // 2, offset), line, font=font, fill=(0,0,0))
        offset += line_h

    if filepath:
        image.save(filepath)
        
    return image




text_on_img(text="new labe")

In [ ]:
#exporti

try:
    from collections.abc import Iterable
except ImportError:
    from collections import Iterable
    
def flatten(lis):
    for item in lis:
            if isinstance(item, Iterable) and not isinstance(item, str):
                for x in flatten(item):
                    yield x
            else:        
                yield item

In [ ]:
list(flatten([1, 2, [3, 4], 5]))

In [ ]:
assert (list(flatten([1,2,[3,4,5, [6,7]], 8, [9, 10]])) == list(range(1, 11)))

In [ ]:
#exporti

def reconstruct_class_images(label_dir, annotation_file, lbl_w=None, lbl_h=None):
    with open(annotation_file) as json_file:
        data = json.load(json_file)
        unique_classes = set(flatten(data.values())) # ipyannotator format
    
    for cl_name in unique_classes:
        if cl_name is None:
            cl_name = "None"
            
        cl_im_name = f'{cl_name}.jpg' if not re.findall("([-\w]+\.(?:jpg|png|jpeg))", cl_name, re.IGNORECASE) else cl_name
   
        text_on_img(text = os.path.splitext(cl_name)[0], filepath = label_dir/cl_im_name, lbl_w=lbl_w, lbl_h=lbl_h)

In [ ]:
fol = Path(tmp_dir.name)/'autogenerated'
fol.mkdir(parents=True, exist_ok=True)
reconstruct_class_images(fol, annot_file)

In [ ]:
clas_path = Path(tmp_dir.name)/'autogenerated'
print(clas_path)
!ls {clas_path}

In [ ]:
#exporti

class Im2ImAnnotatorLogic(HasTraits):
    debug_output = Output(layout={'border': '1px solid black'})
    index = Int(0) # state index
    image_path = Unicode() # current image path
    current_im_num = Int()
    disp_number = Int() # number of labels on screen
    label_state = Dict()
    question_value = Unicode()

    
    def __init__(self, project_path, file_name=None, question=None, 
                 image_dir='pics', step_down=False,
                 label_dir=None, results_dir=None, lbl_w=None, lbl_h=None, n_cols=None, n_rows=None):
        
        self.project_path = Path(project_path)
        self.step_down = step_down
        self.image_dir, self.annotation_file_path = setup_project_paths(self.project_path,
                                                                        file_name=file_name,
                                                                        image_dir=image_dir,
                                                                        results_dir=results_dir)
                      
        # artificialy generate labels if no class images given
        if label_dir is None:
            self.label_dir = Path(self.project_path, 'class_autogenerated_' + ''.join(random.sample(str(uuid.uuid4()), 5)))
            self.label_dir.mkdir(parents=True, exist_ok=True)
            
            question = 'Autogenerated classes'
            
            if self.annotation_file_path.exists():
                reconstruct_class_images(self.label_dir, self.annotation_file_path, lbl_w=lbl_w, lbl_h=lbl_h)
            else:
                text_on_img(text = 'None', filepath = self.label_dir /'None.jpg', lbl_w=lbl_w, lbl_h=lbl_h)       
        else:
            self.label_dir = Path(self.project_path, label_dir)
            
            
        # select images and labels only given annotatin file
        if self.annotation_file_path.is_file():
            with self.annotation_file_path.open() as json_file:
                data = json.load(json_file)
                im_names = data.keys()
                unique_labels = set(flatten(data.values()))
            self.image_paths = sorted(im for im in get_image_list_from_folder(self.image_dir) if str(im) in im_names)
            self.labels_files = sorted(im for im in get_image_list_from_folder(self.label_dir, strip_path=True) if str(im) in unique_labels)
        else:
            self.image_paths = sorted(get_image_list_from_folder(self.image_dir))
            self.labels_files = sorted(get_image_list_from_folder(self.label_dir, strip_path=True))
            
        if not self.image_paths:
            raise Exception ("!! No Images to dipslay !!")
        if not self.labels_files:
            print("!! No labels to display !!")
            
        self.current_im_num = len(self.image_paths)
        labels_num = len(self.labels_files)
        
        if n_cols is None:
            if n_rows is None:  # automatic arrange
                self.label_cols = 3
                self.label_rows = ceil(labels_num / self.label_cols)
            else:  # calc cols to show all labels
                self.label_rows = n_rows
                self.label_cols = ceil(labels_num / self.label_rows)
                
        else: 
            if n_rows is None:  # calc rows to show all labels
                self.label_cols = n_cols
                self.label_rows = ceil(labels_num / self.label_cols) 
            else:  # user defined
                self.label_cols = n_cols
                self.label_rows = n_rows
        
        if (self.label_cols * self.label_rows < labels_num):
            print("!! Not all labels shown. Check n_cols, n_rows args !!")
        
        self.annotations = AnnotationStorage(self.image_paths, dir_in_label=step_down)
        
        if self.annotation_file_path.exists():
            self.annotations.load(self.annotation_file_path)
        else:
            self.annotations.save(self.annotation_file_path)
        
        if question:
            self.question_value = f'<center><p style="font-size:20px;">{question}</p></center>'
        
        
    def _update_im(self):
        self.image_path = str(self.image_paths[self.index])
        
        
    def _update_state(self, change=None): # from annotations 
        if not self.image_path:
            return
        
        if self.image_path in self.annotations:
            current_annotation = self.annotations[self.image_path]
            self.label_state = _storage_format_to_label_state(storage_format=current_annotation or [],
                                                              label_names=self.labels_files,
                                                              label_dir=self.label_dir)


    def _update_annotations(self, index): # from screen
        if self.image_path:
            self.annotations[self.image_path] = _label_state_to_storage_format(self.label_state)
        
    def _save_annotations(self, *args, **kwargs): # to disk
        index = kwargs.pop('old_index', self.index)
        self._update_annotations(index)          
        self.annotations.save(self.annotation_file_path)
        
        
    @observe('index')
    def _idx_changed(self, change):
        ''' On index change save old state 
            and update current state for visualisation
        '''
        self._save_annotations(old_index = change['old'])
        # update new screen
        self._update_im()
        self._update_state()

        
    @debug_output.capture(clear_output=False)
    def _handle_grid_click(self, event, name=None):
        label_changed = Path(self.label_dir,  name)
        
        if label_changed.is_dir():
            # button without image - invalid
            return
        
        label_changed = str(label_changed)
        current_label_state = self.label_state.copy()
        
        # inverse state
        current_label_state[label_changed] = {'answer': not self.label_state[label_changed].get('answer', False)}
        
        # change traitlets.Dict entirely to have change events issued
        self.label_state = current_label_state
        

    def _handle_client_ready(self):
        self._update_im()
        self._update_state()

In [ ]:
i_ = Im2ImAnnotatorLogic(project_path='../data/projects/im2im1/', label_dir='class_images')
# cal.disp_number = 9 # should be synced from gui 
i_.image_dir, i_.annotation_file_path, i_.label_dir, i_.current_im_num, 

##### (Next-> button emulation) 
Increment index to initiate annotation save and switch state for a new screen

In [ ]:
i_.index=2
display(i_.label_state)

In [ ]:
#export

class Im2ImAnnotator(Im2ImAnnotatorGUI):
    
    def __init__(self, project_path, file_name=None, image_dir=None, step_down=False, label_dir=None, results_dir=None, 
                 im_width=100, im_height=100, label_width=150, label_height=150, 
                 n_rows=None, n_cols=None, label_autosize=False, question=None):
        
        self._model = Im2ImAnnotatorLogic(project_path=project_path, file_name=file_name, question=question, 
                                           image_dir=image_dir, step_down=step_down,
                                           label_dir=label_dir, results_dir=results_dir,
                                           lbl_w=label_width, lbl_h=label_height,
                                           n_rows=n_rows, n_cols=n_cols)
        
        
        super().__init__(im_width, im_height, 
                         label_width, label_height, 
                         n_rows = self._model.label_rows, n_cols = self._model.label_cols, 
                         label_autosize = label_autosize)
        
        self._save_btn.on_click(self._model._save_annotations)        

        self._grid_box.on_click(self._model._handle_grid_click)
            
        # set correct slider max value based on image number
        dlink((self._model, 'current_im_num'), (self._navi.model, 'max_im_number'))
        
        # link current image index from controls to annotator model 
        link((self._navi.model, 'index'), (self._model, 'index'))
        
        # link annotation question 
        link((self._model, 'question_value'), (self._grid_label, 'value'))
        
        #link image vizualizer
        dlink((self._model, 'image_path'), (self._image, 'image_path'))
        
        # draw current image and bbox only when client is ready
        self.on_client_ready(self._model._handle_client_ready)
        
        # link state of model and grid box visualizer
        link((self._model, 'label_state'), (self._grid_box, 'current_state'))
        
            
    def to_dict(self, only_annotated=True):
        return self._model.annotations.to_dict(only_annotated)

In [ ]:
im2im = Im2ImAnnotator(project_path='../data/projects/im2im1', image_dir='pics',
#                         label_dir='class_images',
                        label_dir=None,
                        results_dir=None,
                        im_width=300, im_height=300, 
                        label_width=150, label_height=50,
                        n_cols=2, 
#                         n_rows=3,
#                         label_autosize=True, 
                        question="HelloWorld")

im2im

In [ ]:
im2im.to_dict()

In [ ]:
im2im._model.debug_output

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()